C Conversion
---
All we need to do to convert our Mahalanobis Distance model to C is to save the mean vector and inverse of the covariance matrix as constant arrays.

In [23]:
from os import listdir
from os.path import join
import numpy as np
import scipy as sp
from scipy import stats
import c_writer

In [24]:
# Print versions
!python --version
print('Numpy ' + np.__version__)
print('SciPy ' + sp.__version__)

Python 3.7.6
Numpy 1.18.1
SciPy 1.4.1


In [25]:
# Settings (define file locations)
models_path = 'models'  # Where we can find the model files (relative path location)
md_file_name = 'md_model-moving'   # Mahalanobis Distance model arrays (.npz will be added)
c_model_name = 'md_model-moving'   # Will be given .h suffix

c_mean_name = 'model_mu'
c_inv_cov_name = 'model_inv_cov'

In [26]:
# Load model
with np.load(join(models_path, md_file_name) + '.npz') as data:
    model_mu = data['model_mu']
    model_cov = data['model_cov']
print(model_mu)
print(model_cov)

[0.00769517 0.00414957 0.00490948]
[[1.43478461e-05 7.25720447e-06 1.87261642e-06]
 [7.25720447e-06 1.41098698e-05 3.71337361e-06]
 [1.87261642e-06 3.71337361e-06 2.22039820e-06]]


In [27]:
# Calculate inverse of covariance matrix (as it's constant)
inv_cov = sp.linalg.inv(model_cov)
print(inv_cov)

[[  94214.38738546  -49201.81370746    2827.10913737]
 [ -49201.81370746  152282.54724185 -213180.62124189]
 [   2827.10913737 -213180.62124189  804506.68775838]]


In [6]:
# Create constant C arrays for model (mu and inverse covariance)
c_mu = c_writer.create_array(model_mu, 'float', c_mean_name)
c_inv_cov = c_writer.create_array(inv_cov, 'float', c_inv_cov_name)

In [7]:
# Construct header file
header_str = c_writer.create_header(c_mu + '\n' + c_inv_cov, c_model_name)
print(header_str)

#ifndef MD_MODEL-MOVING_H
#define MD_MODEL-MOVING_H

const unsigned int model_mu_dim1 = 3;

const float model_mu[3] = {
    0.007695169087292815, 0.004149572143093924, 0.004909478363535906
};

const unsigned int model_inv_cov_dim1 = 3;
const unsigned int model_inv_cov_dim2 = 3;

const float model_inv_cov[3][3] = {
    94214.3873854578, -49201.81370745852, 2827.1091373699887, 
    -49201.813707458496, 152282.54724185396, -213180.62124189397, 
    2827.109137369931, -213180.6212418939, 804506.6877583836
};

#endif //MD_MODEL-MOVING_H


In [8]:
# Save C header file
with open(join(models_path, c_model_name) + '.h', 'w') as file:
    file.write(header_str)

Save Samples for Testing
---
Convert raw sample to constant C array for testing.

In [8]:
# Saved Numpy test samples file location
sample_file_path = '..\\test_samples'
sample_file_name = 'normal_anomaly_samples'  # Will be given .npz suffix
c_normal_sample_name = 'normal_sample'       # Will be given .h suffix for file
c_anomaly_sample_name = 'anomaly_sample'     # Will be given .h suffix for file

In [9]:
# Load test samples
with np.load(join(sample_file_path, sample_file_name) + '.npz') as data:
    normal_sample = data['normal_sample']
    anomaly_sample = data['anomaly_sample']
print(normal_sample.shape)
print(anomaly_sample.shape)
print(normal_sample[:5])
print(anomaly_sample[:5])

(200, 3)
(200, 3)
[[0.07666  0.170898 0.981445]
 [0.063477 0.169434 0.988281]
 [0.073242 0.166992 0.989258]
 [0.07373  0.170898 0.987305]
 [0.069336 0.166992 0.989746]]
[[-0.024414  0.187988  0.961914]
 [-0.01416   0.195801  0.986816]
 [-0.020996  0.193359  0.98877 ]
 [-0.021973  0.191406  0.983398]
 [-0.010742  0.199219  0.996582]]


In [10]:
# Create C array out of normal sample
c_sample = c_writer.create_array(normal_sample, 'float', c_normal_sample_name)
header_str = c_writer.create_header(c_sample, c_normal_sample_name)
#print(header_str)

In [36]:
# Save C header file with normal sample
with open(join(sample_file_path, c_normal_sample_name) + '.h', 'w') as file:
    file.write(header_str)

In [37]:
# Create C array out of anomaly sample
c_sample = c_writer.create_array(anomaly_sample, 'float', c_anomaly_sample_name)
header_str = c_writer.create_header(c_sample, c_anomaly_sample_name)
print(header_str)

#ifndef ANOMALY_SAMPLE_H
#define ANOMALY_SAMPLE_H

const unsigned int anomaly_sample_dim1 = 200;
const unsigned int anomaly_sample_dim2 = 3;

const float anomaly_sample[200][3] = {
    -0.024414, 0.187988, 0.961914, -0.01416, 0.195801, 0.986816, -0.020996, 
    0.193359, 0.98877, -0.021973, 0.191406, 0.983398, -0.010742, 0.199219, 
    0.996582, -0.029297, 0.183594, 0.976074, -0.006348, 0.19873, 0.988281, 
    -0.022949, 0.18457, 0.978516, -0.015625, 0.161133, 0.975098, -0.020996, 
    0.194824, 0.98584, -0.024902, 0.187988, 0.98291, -0.019531, 0.190918, 
    0.979981, -0.021973, 0.20166, 0.982422, -0.029785, 0.188965, 0.966797, 
    -0.015625, 0.199707, 0.983887, -0.03125, 0.181152, 0.973145, -0.01709, 
    0.193848, 0.973145, -0.022949, 0.194336, 0.985352, -0.024902, 0.193359, 
    0.977051, -0.013672, 0.201172, 0.983398, -0.033203, 0.186035, 0.976074, 
    -0.011719, 0.197266, 0.982422, -0.027832, 0.189453, 0.979981, -0.014648, 
    0.183105, 0.972656, -0.013672, 0.193848, 0.98584, 

In [38]:
# Save C header file with normal sample
with open(join(sample_file_path, c_anomaly_sample_name) + '.h', 'w') as file:
    file.write(header_str)

Develop C functions
---
We don't have access to an easy/efficient library for calculating the Mahalanobis Distance in C, so we need to write a few functions from scratch. It's easiest to get the algorithms down in Python first so we can compare the output from a few samples to known good algorithms.

In [11]:
# Set some settings to test out our functions
sensor_sample_rate = 200    # Hz
sample_time = 0.64           # Time (sec) length of each sample
max_measurements = int(sample_time * sensor_sample_rate)

In [12]:
# Test extracting features (median absolute deviation) using SciPy
sample = normal_sample[0:max_measurements]                  # Truncate to 128 measurements
sp_test_features = stats.median_absolute_deviation(sample)  # Calculate MAD
print(sp_test_features)

[0.00615279 0.00217201 0.00398226]


In [13]:
# Find median in an array (we'll use qsort in C)
# We want to calculate MAD in a C-like fashion to make it easier to port
def median(arr):
    n = len(arr)
    sorted_arr = np.sort(arr)
    if n % 2 == 0:
        median = (sorted_arr[int((n - 1) / 2)] + sorted_arr[int(n / 2)]) / 2.0
    else:
        median = sorted_arr[n / 2]
    return median

In [14]:
# Calculate MAD manually
def calc_mad(arr):
    
    # Find number of elements
    n = len(arr)
    
    # Get median of array
    med = median(arr)

    # Calculate absolute deviation from median for each element
    devs = [0] * n
    for i in range(n):
        devs[i] = abs(arr[i] - med)
    
    # Find median of deviations
    return median(devs)

In [15]:
# Wrapper to extract MAD features for all axis
def extract_mad_features(sample, scale=1.4826):
    mads = []
    
    # Calculate MAD for each axis (note: SciPy scales by 1.4826)
    # https://github.com/scipy/scipy/issues/11090
    for axis in range(sample.shape[1]):
        mads.append(scale * calc_mad(sample[:, axis]))
        
    return mads

In [16]:
# Test MAD algorithm and compare to SciPy answer
sample = normal_sample[0:max_measurements]                  # Truncate to 128 measurements
normal_x = extract_mad_features(sample)
print(normal_x)

[0.006152790000000001, 0.0021720089999999912, 0.00398226359999995]


In [17]:
# Test: calculate Mahalanobis distance using Numpy
x_minus_mu = normal_x - model_mu
left_term = np.dot(x_minus_mu, inv_cov)
mahal = np.dot(left_term, x_minus_mu.T)
print(mahal)

0.4374791356307415


In [18]:
# Calculate the dot product of two vectors
def dot_product(a, b):
    sum = 0
    for i in range(len(a)):
        sum += a[i] * b[i]
        
    return sum

In [19]:
# Do matrix multiplication
def matrix_multiply(a, b):
    
    # Find number of rows and columns
    a_rows = len(a)
    a_cols = len(a[0])
    b_rows = len(b)
    b_cols = len(b[0])
    
    # Check to make sure we can multiply the matrices
    if a_cols != b_rows:
        print('Dimension mismatch')
        return [[]]
    
    # Create return matrix
    prod = [[0 for i in range(b_cols)] for j in range(a_rows)]
    
    # Calculate the dot product for each element in the return matrix
    for i in range(a_rows):
        for j in range(b_cols):
            for k in range(a_cols):
                prod[i][j] += a[i][k] * b[k][j]
    
    return prod

In [20]:
# Calculate Mahalanobis distance the C way (with const inverse covariance)
def mahalanobis(x, mu, inv_cov):
    
    # Subtract each element in X from the mean
    x_minus_mu = [0 for i in range(len(x))]
    for i in range(len(x)):
        x_minus_mu[i] = x[i] - mu[i]
        
    # Compute product of prev term and inverse covariance
    left_term = matrix_multiply([x_minus_mu], inv_cov)
    
    # Transpose difference matrix
    x_minus_mu_t = [[i] for i in x_minus_mu]
    
    # Matrix multiply prev term and difference
    mahal = matrix_multiply(left_term, x_minus_mu_t)
    
    return mahal[0][0]

In [21]:
# Test Mahalanobis function with normal sample
print('Normal Sample')
print('MAD:', normal_x)
print('Mahalanobis distance:', mahalanobis(normal_x, model_mu.tolist(), inv_cov.tolist()))

Normal Sample
MAD: [0.006152790000000001, 0.0021720089999999912, 0.00398226359999995]
Mahalanobis distance: 0.4374791356307415


In [22]:
# Test Mahalanobis function with anomaly sample
print('Anomaly Sample')
sample = anomaly_sample[0:max_measurements]  # Truncate to 128 measurements
anomaly_x = extract_mad_features(sample)
print('MAD:', anomaly_x)
print('Mahalanobis distance:', mahalanobis(anomaly_x, model_mu, inv_cov))

Anomaly Sample
MAD: [0.0057917769, 0.0050675268000000065, 0.00542928120000008]
Mahalanobis distance: 0.6499213735195706
